# Defensa Técnica: Prompt Engineering para un Chatbot Financiero

## Introducción

He desarrollado un sistema de prompt engineering para un chatbot financiero, utilizando FastAPI como framework, LangChain como orquestador y Google Gemini como modelo de lenguaje. El objetivo principal fue construir un sistema capaz de comprender y responder a consultas financieras de manera precisa, estructurada y empática, validado a través de un sistema de evaluación automatizado.

## Estrategia de Prompt Engineering

Mi estrategia se centró en la creación de un único y robusto prompt que integra varias técnicas para guiar al modelo de forma efectiva. Las técnicas clave implementadas son:

1.  **Role-Based Prompting**: Asigné al modelo el rol de "Álex", un asistente financiero experto de ITTI, con un tono y una personalidad definidos para asegurar consistencia.
2.  **Few-Shot Learning**: Incluí ejemplos de interacciones (pregunta-respuesta) en el prompt para mostrarle al modelo el formato de salida esperado, lo que reduce la probabilidad de errores de formato.
3.  **Structured Output**: Exigí respuestas en formato JSON y utilicé modelos de Pydantic para validar la estructura, asegurando que la salida del modelo sea predecible y fácil de procesar.
4.  **In-Context Learning**: Proporcioné información detallada sobre los productos directamente en el prompt, lo que permite al modelo responder con datos precisos sin depender de conocimiento externo.

### Justificación de la Estructura del Prompt

Se optó por un único prompt en formato XML por varias razones técnicas. Primero, la estructura jerárquica de XML permite una delimitación clara y explícita de las distintas secciones del prompt (`<role>`, `<guidelines>`, `<output_format>`, `<examples>`), lo que mejora la capacidad del modelo para interpretar las instrucciones. Segundo, centralizar la lógica en un solo artefacto simplifica la gestión y el versionado en el contexto de una Prueba de Concepto (POC). Este enfoque monolítico, aunque simple, demuestra ser robusto para guiar al modelo sin la complejidad de un sistema multi-prompt o basado en agentes para este caso de uso inicial.

## Arquitectura del Sistema

La arquitectura está diseñada para ser modular y escalable:

-   **FastAPI**: Provee la interfaz de la API, aprovechando la validación de datos nativa con Pydantic.
-   **LangChain**: Actúa como una capa de abstracción que facilita la comunicación con diferentes modelos de lenguaje, permitiendo intercambiar proveedores (Google, OpenAI) con mínimos cambios.
-   **Servicios Desacoplados**: La lógica de negocio está separada en servicios específicos para la gestión de prompts, la interacción con el LLM y la evaluación, lo que facilita el mantenimiento y la escalabilidad.
-   **Modelos Pydantic**: Garantizan que tanto las entradas como las salidas de la API cumplan con un esquema definido, añadiendo una capa de robustez al sistema.

## Sistema de Evaluación: Metodología y Métricas

Para validar la efectividad del prompt, se implementó un framework de evaluación automatizado (`run_evaluation.py`) que ejecuta el sistema contra un dataset de prueba y calcula un conjunto de métricas objetivas.

### Dataset de Evaluación

Se construyó un dataset sintético (`evaluation_dataset.csv`) diseñado específicamente para esta POC. Cada registro contiene:
- `user_query`: Una pregunta representativa que un cliente podría formular.
- `ideal_response`: Una respuesta redactada por un experto, considerada el "gold standard" para la evaluación de similitud.
- `expected_intent`: La intención correcta que el modelo debe identificar.
- `expected_product`: El producto correcto que el modelo debe identificar.

Este dataset permite una evaluación controlada y reproducible de las capacidades fundamentales del sistema: la comprensión de la consulta y la calidad de la respuesta.

### Métricas de Evaluación

La performance del sistema se mide con un enfoque multi-faceta, combinando métricas de clasificación y de similitud semántica.

1.  **Accuracy de Clasificación (Intención y Producto)**: Esta es una métrica fundamental que valida si el modelo comprende la consulta del usuario a un nivel básico. Se calcula como el porcentaje de predicciones correctas sobre el total de casos. Un alto accuracy aquí es un prerrequisito para una interacción exitosa.

2.  **Similitud Semántica (Cosine Similarity)**: En lugar de métricas léxicas como BLEU o ROUGE, se eligió la similitud de coseno sobre embeddings de oraciones. La razón es que la similitud semántica evalúa el **significado y la relevancia** del contenido, no solo el solapamiento de palabras. Esto es crucial para un chatbot que debe ofrecer respuestas coherentes y con sentido, aunque no use exactamente las mismas palabras que la respuesta ideal. La implementación utiliza el framework `Sentence-Transformers`, basado en el trabajo de Reimers & Gurevych, que ha demostrado ser altamente eficaz para generar embeddings semánticamente ricos.

    - *Referencia*: Reimers, N., & Gurevych, I. (2019). *Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks*.

3.  **Métricas Cualitativas (LLM-as-a-Judge)**: Para medir aspectos más sutiles como la **empatía, claridad y tono profesional**, se utiliza un enfoque de "LLM como juez". En este método, se realiza una segunda llamada a un LLM con un prompt específico para que evalúe la respuesta generada en base a estos criterios cualitativos. Este enfoque permite una evaluación escalable de la calidad de la respuesta que va más allá de la simple corrección factual.

Este sistema de evaluación trimodal (clasificación, similitud y juicio cualitativo) proporciona una visión integral y robusta del rendimiento del prompt.

## Conclusión

Esta Prueba de Concepto demuestra la viabilidad de utilizar un único prompt bien estructurado para controlar un LLM en una tarea de chatbot financiero. La combinación de técnicas de prompt engineering y un framework de evaluación robusto ha permitido desarrollar un sistema capaz de comprender las consultas de los usuarios, generar respuestas precisas y mantener un tono profesional. Los resultados de la evaluación validan que el enfoque adoptado es sólido y proporciona una base excelente para futuras iteraciones y mejoras del sistema.